In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
num_epochs = 15

# 데이터셋 전처리 및 데이터 증강
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),      # 랜덤 회전 추가
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# CIFAR-10 데이터셋 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의 (ResNet-50 사용)
model = models.resnet50(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256), # 레이어 추가
    nn.ReLU(),
    nn.Dropout(0.5),                     # 드롭아웃 추가
    nn.Linear(256, 10)
)

# GPU 사용 가능하면 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수, 옵티마이저, 학습률 스케줄러 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # 학습률을 에포크마다 절반으로 감소

# 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 옵티마이저 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(train_loader)

# 테스트 함수
def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # 예측 및 정확도 계산
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 모델 학습 및 테스트
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    test_accuracy = test(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    scheduler.step()  # 학습률 업데이트

print("모델 학습 완료!")

Files already downloaded and verified
Files already downloaded and verified


c:\ProgramData\Anaconda3\envs\mjjenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\mjjenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/15], Loss: 1.9459, Test Accuracy: 0.3385
Epoch [2/15], Loss: 1.7692, Test Accuracy: 0.3429
Epoch [3/15], Loss: 1.7560, Test Accuracy: 0.3652
Epoch [4/15], Loss: 1.6486, Test Accuracy: 0.4373
Epoch [5/15], Loss: 1.6147, Test Accuracy: 0.4388
Epoch [6/15], Loss: 1.5124, Test Accuracy: 0.5167
Epoch [7/15], Loss: 1.4079, Test Accuracy: 0.5465
Epoch [8/15], Loss: 1.3826, Test Accuracy: 0.5830
Epoch [9/15], Loss: 1.2698, Test Accuracy: 0.6197
Epoch [10/15], Loss: 1.2125, Test Accuracy: 0.6316
Epoch [11/15], Loss: 1.1069, Test Accuracy: 0.6702
Epoch [12/15], Loss: 1.0630, Test Accuracy: 0.6782
Epoch [13/15], Loss: 1.0265, Test Accuracy: 0.6796
Epoch [14/15], Loss: 0.9877, Test Accuracy: 0.6877
Epoch [15/15], Loss: 0.9571, Test Accuracy: 0.7030
모델 학습 완료!
